## COGS 108 - Final Project

## Part 0: Setup

In [2]:
# Imports
import pandas as pd
import numpy as np

## Part 1: Data Cleaning

## 1a) Condense dat files: block hash, transaction overview
Drop irrelevant columns in bh.dat and tx.dat; store result dataframes to bh.dat and tx.dat.

Result:

bh.dat: 

| Block ID | Block Timestamp |

tx.dat: 

|Transaction ID | Block ID |

In [ ]:
# file path for data
bh_filepath = 'data/bh.dat'
tx_filepath = 'data/tx.dat'

# read data
df_block_hash = pd.read_csv(bh_filepath, sep = '\t')
df_transaction = pd.read_csv(tx_filepath, sep = '\t')

# set column names
df_block_hash.columns = ['block ID', 'hash', 'timestamp', 'number of transactions']
df_transaction.columns = ['transaction ID', 'block ID', 'input count', 'output count']

# drop irrelevant columns
df_block_hash.drop(['hash', 'number of transactions'], axis = 1, inplace = True)
df_transaction.drop(['input count', 'output count'], axis = 1, inplace = True)

# check head after drop
print(df_block_hash.head())
print(df_transaction.head())

# write block hash dataframe to a new dat file
bh_drop_filepath = 'data/bh.dat'
df_block_hash.to_csv(path_or_buf = bh_drop_filepath, sep = '\t', index = False, columns = ['block ID', 'timestamp'])

tx_drop_filepath = 'data/tx.dat'
df_transaction.to_csv(path_or_buf = tx_drop_filepath, sep = '\t', index = False, 
                      columns = ['transaction ID', 'block ID'])

## 1b) Condense dat files: transaction input
Split txin.dat to small dat files; drop irrelevant columns; store result dataframes to txin_1.dat to txin_11.dat.

Result:

txin_1.dat - txin_11.dat: 

| Transaction ID | Address ID |

In [ ]:
# split transaction_input file
!split -l 70000000 data/txin.dat t

# set filepath
num_txin_file = 11

txin_input_filepath = ['txin/taa', 'txin/tab', 'txin/tac', 'txin/tad', 'txin/tae', 'txin/taf', 'txin/tag', 'txin/tah',
                      'txin/tai', 'txin/taj', 'txin/tak']
txin_output_filepath = ['txin/txin_1.dat', 'txin/txin_2.dat', 'txin/txin_3.dat', 'txin/txin_4.dat', 'txin/txin_5.dat', 
                        'txin/txin_6.dat', 'txin/txin_7.dat', 'txin/txin_8.dat', 'txin/txin_9.dat', 'txin/txin_10.dat', 
                        'txin/txin_11.dat']

# for loop to drop irrelevant columns in transaction_input
for x in range(num_txin_file):
    curr_input_filepath = txin_input_filepath[x]
    curr_output_filepath = txin_output_filepath[x]
    
    df_txin = pd.read_csv(curr_input_filepath, sep = '\t')
    
    df_txin.columns = ['transaction ID', 'input sequence', 'previous transaction ID', 'previous output sequence', 
                       'address ID', 'sum']
    
    df_txin.drop(['input sequence', 'previous transaction ID', 'previous output sequence', 'sum'], 
                 axis = 1, inplace = True)
    
    df_txin.to_csv(path_or_buf = curr_output_filepath, sep = '\t', index = False, 
                   columns = ['transaction ID', 'address ID'])

## 1c) Condense dat files: transaction output
Split txout.dat to small dat files; drop irrelevant columns; store result dataframes to txout_1.dat to txout_12.dat.

Result:

txout_1.dat - txout_12.dat: 

| Transaction ID | Address ID |

In [ ]:
# split transaction_output file
!split -l 70000000 data/txout.dat m

# set filepath
num_txout_file = 12

txout_input_filepath = ['txout/maa', 'txout/mab', 'txout/mac', 'txout/mad', 'txout/mae', 'txout/maf', 'txout/mag', 
                        'txout/mah', 'txout/mai', 'txout/maj', 'txout/mak', 'txout/mal']
txout_output_filepath = ['txout/txout_1.dat', 'txout/txout_2.dat', 'txout/txout_3.dat', 'txout/txout_4.dat', 
                         'txout/txout_5.dat', 'txout/txout_6.dat', 'txout/txout_7.dat', 'txout/txout_8.dat', 
                         'txout/txout_9.dat', 'txout/txout_10.dat', 'txout/txout_11.dat', 'txout/txout_12.dat']

# for loop to drop irrelevant columns in transaction_output
for x in range(num_txout_file):
    curr_input_filepath = txout_input_filepath[x]
    curr_output_filepath = txout_output_filepath[x]
    
    df_txout = pd.read_csv(curr_input_filepath, sep = '\t')
    
    df_txout.columns = ['transaction ID', 'output sequence', 'address ID', 'sum']
    
    df_txout.drop(['output sequence', 'sum'], axis = 1, inplace = True)
    
    df_txout.to_csv(path_or_buf = curr_output_filepath, sep = '\t', index = False, 
                   columns = ['transaction ID', 'address ID'])

## 1d) Convert UNIX time to standard time
Implement a function that converts UNIX to standard time; add a column "year" in bh.dat.

Before: 

| Block ID | Block Timestamp |

After: 

| Block ID | Block Timestamp | Month | Year |

In [ ]:
from datetime import datetime
file='bh_simplified.dat'

df=pd.read_csv(file, sep='\t')

def getYear(data):
    d=datetime.fromtimestamp(data)
    return d.timetuple().tm_year
def getMonth(data):
    d=datetime.fromtimestamp(data)
    return d.timetuple().tm_mon

df['month']=df['timestamp'].apply(getMonth)
df['year']=df['timestamp'].apply(getYear)

df.to_csv('bh_readable.dat', sep='\t', index=False)

## 1e) Range of Block IDs for each year

In [ ]:
bh_filepath = 'data/bh_readable.dat'
df_block_hash = pd.read_csv(bh_filepath, sep = '\t')
df_block_hash.columns = ['blockID', 'timestamp', 'month', 'year']
df_block_hash.head()

range_block_each_year = {}

target_year = df_block_hash["year"][0]

target_block_start = 1
target_block_end = 1

for row in df_block_hash.itertuples():
    curr_year = row.year
    
    if curr_year > target_year:
        curr_block_id = row.blockID
        
        target_block_end = curr_block_id - 1
        
        range_block_each_year[target_year] = (target_block_start, target_block_end)
        
        target_block_start = curr_block_id
        
        target_year += 1
        
    if target_year == 2018:
        break
        
print(range_block_each_year)

## 1f) Range of Transaction IDs for each year

In [ ]:
tx_filepath = 'data/tx.dat'
df_tx = pd.read_csv(tx_filepath, sep = '\t')
df_tx.columns = ['transactionID', 'blockID']

range_tx_each_year = {}

for key in range_block_each_year:
    value = range_block_each_year[key]
    
    block_range_start = value[0]
    block_range_end = value[1]
    
    df_curr = df_tx[df_tx['blockID'] == block_range_start]
    df_curr.reset_index()
    tx_range_start = df_curr.iloc[0]['transactionID']
    
    df_curr = df_tx[df_tx['blockID'] == block_range_end]
    df_curr.reset_index()
    tx_range_end = df_curr.iloc[len(df_curr) - 1]['transactionID']
    
    range_tx_each_year[key] = (tx_range_start, tx_range_end)
    
print(range_tx_each_year)

## 1g) Re-split tx_in and tx_out dat files to groups of 2-year time periods


Used VIM to split up into 2-year chunks.

## 1h) Range of Address IDs for each year

In [3]:
data2 = pd.read_csv('TxinFiles/txin_2010.dat', sep='\t')
data3 = pd.read_csv('TxinFiles/txin_2011.dat', sep='\t')
data4 = pd.read_csv('TxinFiles/txin_2012.dat', sep='\t')
data5 = pd.read_csv('TxinFiles/txin_2013.dat', sep='\t')
#data6 = pd.read_csv('TxinFiles/txin_2014.dat', sep='\t')
#data7 = pd.read_csv('TxinFiles/txin_2015.dat', sep='\t')
#data8 = pd.read_csv('TxinFiles/txin_2016.dat', sep='\t')
#data9 = pd.read_csv('TxinFiles/txin_2017.dat', sep='\t')

In [4]:
#data8['address ID'] = pd.to_numeric(data8['address ID'], errors='coerce', downcast='integer')

NameError: name 'data8' is not defined

In [6]:
data = [data2, data3, data4, data5]#, data6, data7, data8, data9]
years = ['2010', '2011', '2012', '2013']#, '2014', '2015', '2016', '2017']

In [7]:
max_array = []
for frame in data:
    tx_max = frame['address ID'].max()
    max_array.append(tx_max)

In [8]:
min_array = [data2['address ID'].min()]
for i in range(len(max_array)):
    min_array.append(max_array[i] + 1)

In [9]:
range_id_each_year = {}
for i in range(len(years)):
    range_id_each_year[years[i]] = (min_array[i], max_array[i])
range_id_each_year

{'2010': (171, 176514),
 '2011': (176515, 2769557),
 '2012': (2769558, 8714741),
 '2013': (8714742, 24822602)}

## Part 2: Sample Datasets
Data files at this point:

block hash: bh.dat

transaction overview: tx.dat

transaction input: txin_2010.dat, txin_2012.dat, txin_2014.dat, txin_2016.dat

## 2a) Load the Data
Load data above into dataframes.

In [ ]:
# Code here

## 2b) Initialize the Dataframe
Initialize the following dataframe:

| Address ID | Year | NumTX |

Address ID: Bitcoin account identifier

Year: Year in which the Address ID has its first transaction

NumTx: Number of transactions in total belong to that Bitcoin Address ID

In [ ]:
# Code here

## 2c) Sample Address ID

Randomly Sample 100 address IDs in each transaction input/output dat file.

Update the following columns in the dataframe:

| Address ID | Year |

In [26]:
address = data2['address ID'][data2['address ID'] >= 31617]
address = address[address <= 176514]
address.drop_duplicates()
id_2010 = np.array(address.sample(125))

In [27]:
address = data3['address ID'][data3['address ID'] >= 176515]
address = address[address <= 2769557]
address.drop_duplicates()
id_2011 = np.array(address.sample(125))

In [28]:
id_10_11 = id_2010 + id_2011
df_Addr_Year['Address ID']=id_10_11
df_Addr_Year['Year']='2010'

array([2688364, 2604888, 2354830, 2118417, 2168242, 2218848,  606676,
        436978, 1650823, 1470767, 2641328, 1966173,  988617,  611664,
        563838,  941268,  587853, 1080429,  575618, 1474621, 2083368,
        238119, 2904900,  919756, 2811548,  710861, 2637505, 2460222,
        629703, 1064927,  323912, 1072994, 1525707, 2741607, 2544163,
       2885374,  691351, 2447247,  706861, 2693454, 1261853, 1618593,
       1052601, 1771731,  513967, 1926695,  861076,  780576, 1984759,
        949811,  387932,  657623, 1998872, 2575471, 1989581,  549509,
       2317360, 2252638,  743699,  729812, 2032073,  577120, 2205099,
       2554805, 2247311,  750205,  916036, 1696920, 2206098, 1164741,
       1640148,  431706, 1055307,  518111, 1686486, 1834330, 1694011,
       1397200,  274134, 2840909,  682503,  766155, 1822470, 1319171,
        329409,  637270,  383842, 2230467, 2767987,  640225, 2216759,
       1567641,  954136, 2438525,  325178, 1371240,  519790,  691884,
        852686, 1006

In [19]:
address = data4['address ID'][data4['address ID'] >= 2769558]
address = address[address <= 8714741]
address.drop_duplicates()
id_2012 = np.array(address.sample(125))

In [20]:
address = data5['address ID'][data5['address ID'] >= 8714742]
address = address[address <= 24822602]
address.drop_duplicates()
id_2013 = np.array(address.sample(125))

In [21]:
id_12_13 = id_2012 + id_2013

In [ ]:
address = data6['address ID'][data6['address ID'] >= 24822603]
address = address[address <= 58929852]
address.drop_duplicates()
id_2014 = np.array(address.sample(125))

In [ ]:
address = data7['address ID'][data7['address ID'] >= 58929853]
address = address[address <= 115528140]
address.drop_duplicates()
id_2015 = np.array(address.sample(125))

In [ ]:
id_14_15 = id_2014 + id_2015

In [ ]:
address = data8['address ID'][data8['address ID'] >= 115528141]
address = address[address <= 210026230]
address.drop_duplicates()
id_2016 = np.array(address.sample(125))

In [ ]:
address = data9['address ID'][data9['address ID'] >= 210026231]
address = address[address <= 369453964]
address.drop_duplicates()
id_2017 = np.array(address.sample(125))

In [ ]:
id_16_17 = id_2016 + id_2017

## 2d) Accumulate Number of Transactions

Iterate through all the transaction inputs and outputs; accumulate the numbers of transactions for each sample.

Update the following columns in the dataframe:

| NumTX |

In [24]:
num_tx = {}
data_10_17 = [data2, data3, data4, data5]#, data6, data7, data8, data9]
for user in id_10_11:
    num_tx[user] = 0
    for frame in data_10_17:
        d = frame[frame['address ID'] == user]
        d = d.drop_duplicates()
        num_tx[user] = num_tx[user] + len(d)
num_tx

{486152: 1,
 942928: 1,
 416679: 1,
 985286: 1,
 695041: 1,
 1793692: 1,
 1000015: 1,
 1752253: 1,
 2112794: 0,
 2046752: 1,
 1804584: 1,
 440354: 1,
 781366: 1,
 1117343: 0,
 1750216: 1,
 653716: 1,
 2518275: 1,
 2461230: 1,
 805578: 1,
 1463427: 1,
 884092: 1,
 651639: 1,
 2817705: 1,
 1804806: 1,
 1288027: 1,
 2388499: 1,
 315272: 1,
 1758406: 0,
 1005630: 2,
 2091914: 1,
 1070728: 0,
 523387: 1,
 2323925: 1,
 378690: 1,
 2467026: 1,
 2242953: 1,
 1885331: 0,
 2755845: 1,
 2868645: 1,
 1685893: 1,
 478591: 1,
 1892981: 1,
 1147287: 1,
 1548502: 1,
 1331971: 0,
 913768: 1,
 2692188: 1,
 1498701: 1,
 1019045: 1,
 1071435: 0,
 2033058: 1,
 1738933: 1,
 1104700: 1,
 956110: 1,
 964883: 1,
 2099055: 1,
 2092572: 1,
 2342131: 1,
 1643815: 1,
 1448569: 1,
 2804703: 1,
 895041: 1,
 2301007: 1,
 1028036: 1,
 1779565: 1,
 278400: 1,
 259872: 1,
 762686: 1,
 2337738: 1,
 1571943: 1,
 1462237: 1,
 1471124: 1,
 2511375: 1,
 2432876: 1,
 2195225: 1,
 2076736: 1,
 360707: 1,
 1041241: 1,
 1875577:

## Part 3: Normal Distribution and T-test
Apply Normal Distributions and T-tests to the following 2 groups of samples only:

Group 1:

Samples of Address IDs that have their first transaction in year 2010 or 2011

Group 2:

Samples of Address IDs that have their first transaction in year 2016 or 2017

## 3a) Mean number of transactions
What is the average number of transactions for address IDs from each of the 2 groups?

Get a taste of how different the means are.

In [2]:
# Code here

## 3b) Normal Distribution Test
Check that each of the 2 groups has nearly normal distribution.

In [ ]:
# Code here

## 3c) Compare Number of Samples and Population
Check that the number of samples is < 10 % of the total population.

In [ ]:
# Code here

## 3d) T-test
Apply the T-test to the 2 groups.

In [ ]:
# Code here

## Part 4: Analysis of Variance/ANOVA
Apply ANOVA to all the 4 groups of samples:

Group 1:

Samples of Address IDs that have their first transaction in year 2010 or 2011

Group 2:

Samples of Address IDs that have their first transaction in year 2012 or 2013

Group 3:

Samples of Address IDs that have their first transaction in year 2014 or 2015

Group 4:

Samples of Address IDs that have their first transaction in year 2016 or 2017

## 4a) Normal Distribution Test
Check that each group has nearly normal distribution.

In [ ]:
# Code here

## 4b) Compare Number of Samples and Population
Check that the number of samples is < 10 % of the total population.

In [ ]:
# Code here

## 4c) ANOVA test
Apply ANOVA to the 4 groups of samples.

In [ ]:
# Code here

## Part 5: Linear Model
Apply a Linear Model to the data.

Independent variable:

time: month and year when the address ID has its first transaction

Dependent variable:

NumTx: Number of total transactions for each address ID

## 5a) Create Linear Model

In [ ]:
# Code here

## 5b) Correlation Coefficient

In [ ]:
# Code here

## 5c) Predict Number of Transactions from Age of Address ID

In [ ]:
# Code here

## Part 6: Further Analysis